In [ ]:
#from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
#from google.colab import auth
from google.colab import drive
drive.mount('/content/drive')

import json
from datetime import datetime
import pandas as pd
import numpy as np
import csv
import re # regular expressinons
import openpyxl # library to work with excel docs
from pprint import pprint

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Measures Matching

Using the [Weights_Measures](https://docs.google.com/spreadsheets/d/1Hu-66QM8Nat4E3xBLgesX2rmt2qin1yti3ZmvxDhZcU/edit?usp=sharing) equivalency table, we will first make a dictionary of units. Then we can match each 'form' in the words-df (above) to the value in 'transliteration' in the table.

1. Match each unit found in the 'form' of a text.
  * Unit types include: area, weight, capacity, length, count
2. Obtain the count of that unit (always labeled 'n' in POS)
  * Make the conversion to modern equivalent
3. Obtain the asset associated with the unit (1 or 2 tokens before / after the unit, always a Noun 'N')
  * We will want to make a list of all assets counted for each text
4. For each 'unit_tr' with an integer in the parentheses, e.g. (BAN₂), we want to provide two formats, so that we can use our search function on texts with different formats (upper / lower / standard int / unicode int):
  * (BAN₂)
  * (BAN2)
  * (ban₂)
  * (ban2)

### Dictionary of Measurements

1. Measurement Conversion: this dictionary will be useful for making the conversion to the modern equivalents for area, weight, capacity, length, and counts.

|unit_cu|unit_tr|nr|unit|type|
|-------|-------|--|-----|----|
|𒁹𒋡     |1 SILA₃|1|litre|capacity|
|𒑏     |1(BAN₂)|10|litre|capacity|
|𒁹𒋗𒋛  |1 ŠU.SI|0.017|meter|length|
|𒁹𒊺    |1 ŠE|0.05 |gram|weight|
|𒁹     |1(diš)|1|number|count|
|𒈫    |2(diš)|2|number|count|
|𒁹𒊺   |1 ŠE|0.2|sq_meter|area|


2. [Optional] Measurement Key Value Pair Search: We need to make a separate dictionary of the unique 'unit_tr' without the numbers, for mathcing purposes. Some of the units retain the number in the transliteration (i.e. those in parentheses), and others do not.

|unit_cu|unit_tr|value|unit|type|
|-------|-------|-----|----|-----|
|𒋡|SILA₃|1|litre|capacity|
|𒑏|1(BAN₂)|10|litre|capacity|
|𒋗𒋛|ŠU.SI|0.017|meter|length|
|𒊺|ŠE|0.05|gram|weight|
|𒁹|1(diš)|1|number|count|

* Note: some signs are used for multiple different units, e.g. še can be either wight or sq_meters.

In [ ]:
path = "/content/drive/MyDrive/FactGrid Cuneiform (AWCA)/Econ/Weights_Measures.xlsx"
weights_excel = pd.ExcelFile(path)

In [ ]:
# We are reading and downloading our excel documents
# From the files: [area, weight, capacity, length, count]

area = pd.read_excel(weights_excel, 'Area')
weight = pd.read_excel(weights_excel, 'Weight')
capacity = pd.read_excel(weights_excel, 'Capacity')
length = pd.read_excel(weights_excel, 'Length')
count = pd.read_excel(weights_excel, 'Count')

# Selecting the column we need from the excel documents

area = area.loc[:, ['Cuneiform', 'Transliteration', 'm²', 'unit']]
weight = weight.loc[:, ['Cuneiform', 'Transliteration', 'grams', 'unit']]
capacity = capacity.loc[:, ['Cuneiform', 'Transliteration', 'Approximate modern value (litres)', 'unit']]
length = length.loc[:, ['Cuneiform', 'ORACC Transliteration', 'm', "unit"]]
count = count.loc[:, ['Cuneiform', 'Transliteration', 'value (of diš = 1)', 'unit']]

In [ ]:
# Because we have columns with different names
# We want to make sure all of them share the same names

area.rename(columns={'Cuneiform': 'unit_cu', 'Transliteration': 'unit_tr', 'm²': 'nr'}, inplace=True)
weight.rename(columns={'Cuneiform': 'unit_cu', 'Transliteration': 'unit_tr', 'grams': 'nr'}, inplace=True)
capacity.rename(columns={'Cuneiform': 'unit_cu', 'Transliteration': 'unit_tr', 'Approximate modern value (litres)': 'nr'}, inplace=True)
length.rename(columns={'Cuneiform': 'unit_cu', 'ORACC Transliteration': 'unit_tr', 'm': 'nr'}, inplace=True)
count.rename(columns={'Cuneiform': 'unit_cu', 'Transliteration': 'unit_tr', 'value (of diš = 1)': 'nr'}, inplace=True)

In [ ]:
# Adding a new column "Type"

static_value = "area"
area['Type'] = static_value

static_value = "weight"
weight['Type'] = static_value

static_value = "capacity"
capacity['Type'] = static_value

static_value = "length"
length['Type'] = static_value

static_value = "count"
count['Type'] = static_value

In [ ]:
# Joining the 5 dataframes into one

new_df = pd.concat([area, weight, capacity, length, count], axis = 0)
new_df = new_df.reset_index(drop=True)

In [ ]:
new_df

,unit_cu,unit_tr,nr,unit,Type
0,𒃷,GANA₂,3600.0,sq_meter,area
1,𒃷,GAN₂,3600.0,sq_meter,area
2,𒁹𒊺,1 ŠE,0.2,sq_meter,area
3,𒁹𒂆,1 GIN₂,0.6,sq_meter,area
4,𒁹𒊬,1 SAR,36.0,sq_meter,area
...,...,...,...,...,...
282,𒐮,3(šar'u),108000,number,count
283,𒐰,4(šar'u),144000,number,count
284,𒐱,5(šar'u),180000,number,count
285,𒊹𒃲,1(šargal){gal},216000,number,count


In [ ]:
# The specific pattern we want to extract from the string
pattern = r'([a-zA-Z]š?)(\d)'
pattern_uni = r'[\u2080-\u2089]'

unicode_dic = {'0': '₀',
                 '1': '₁',
                 '2': '₂',
                 '3': '₃',
                 '4': '₄',
                 '5': '₅',
                 '6': '₆',
                 '7': '₇',
                 '8': '₈',
                 '9': '₉'}

digit_dic = { '₀':'0',
             '₁': '1',
             '₂': '2',
             '₃': '3',
             '₄': '4',
             '₅': '5',
             '₆': '6',
             '₇': '7',
             '₈': '8',
             '₉': '9'}

# function to replace regular digit for the unicode
def replace_pattern(match):
    letter = match.group(1)
    digit = match.group(2)
    if digit in unicode_dic:
        return letter + unicode_dic[digit]
    else:
        return match.group(0)

# function to replace unicode for the regular digit
def replace_unicode(match):
    subscript = match.group(0)
    return digit_dic.get(subscript, subscript)
new_df['unit_tr_unicode'] = new_df['unit_tr'].apply(lambda x: re.sub(pattern, replace_pattern, x))
new_df['unit_tr_unicode'] = new_df['unit_tr_unicode'].str.upper()
new_df['unit_tr_digit'] = new_df['unit_tr'].apply(lambda x: re.sub(pattern_uni, replace_unicode, x))
new_df['unit_tr_digit'] = new_df['unit_tr_digit'].str.upper()

new_df['unit_tr_unicode_lower'] = new_df['unit_tr_unicode'].str.lower()
new_df['unit_tr_digit_lower'] = new_df['unit_tr_digit'].str.lower()
new_df.head(20)

,unit_cu,unit_tr,nr,unit,Type,unit_tr_unicode,unit_tr_digit,unit_tr_unicode_lower,unit_tr_digit_lower
0,𒃷,GANA₂,3600.0,sq_meter,area,GANA₂,GANA2,gana₂,gana2
1,𒃷,GAN₂,3600.0,sq_meter,area,GAN₂,GAN2,gan₂,gan2
2,𒁹𒊺,1 ŠE,0.2,sq_meter,area,1 ŠE,1 ŠE,1 še,1 še
3,𒁹𒂆,1 GIN₂,0.6,sq_meter,area,1 GIN₂,1 GIN2,1 gin₂,1 gin2
4,𒁹𒊬,1 SAR,36.0,sq_meter,area,1 SAR,1 SAR,1 sar,1 sar
5,𒑟,1/8(IKU),450.0,sq_meter,area,1/8(IKU),1/8(IKU),1/8(iku),1/8(iku)
6,𒑠,1/4(IKU),900.0,sq_meter,area,1/4(IKU),1/4(IKU),1/4(iku),1/4(iku)
7,𒀹,1/2(IKU),1800.0,sq_meter,area,1/2(IKU),1/2(IKU),1/2(iku),1/2(iku)
8,𒀸,1(IKU),3600.0,sq_meter,area,1(IKU),1(IKU),1(iku),1(iku)
9,𒐀,2(IKU),7200.0,sq_meter,area,2(IKU),2(IKU),2(iku),2(iku)


In [ ]:
unit_df = new_df.iloc[5:,:]
unit_df.head(20)

,unit_cu,unit_tr,nr,unit,Type,unit_tr_unicode,unit_tr_digit,unit_tr_unicode_lower,unit_tr_digit_lower
5,𒑟,1/8(IKU),450.0,sq_meter,area,1/8(IKU),1/8(IKU),1/8(iku),1/8(iku)
6,𒑠,1/4(IKU),900.0,sq_meter,area,1/4(IKU),1/4(IKU),1/4(iku),1/4(iku)
7,𒀹,1/2(IKU),1800.0,sq_meter,area,1/2(IKU),1/2(IKU),1/2(iku),1/2(iku)
8,𒀸,1(IKU),3600.0,sq_meter,area,1(IKU),1(IKU),1(iku),1(iku)
9,𒐀,2(IKU),7200.0,sq_meter,area,2(IKU),2(IKU),2(iku),2(iku)
10,𒐁,3(IKU),10800.0,sq_meter,area,3(IKU),3(IKU),3(iku),3(iku)
11,𒐻,3(IKU)@v,10800.0,sq_meter,area,3(IKU)@V,3(IKU)@V,3(iku)@v,3(iku)@v
12,𒐂,4(IKU),14400.0,sq_meter,area,4(IKU),4(IKU),4(iku),4(iku)
13,𒐃,5(IKU),18000.0,sq_meter,area,5(IKU),5(IKU),5(iku),5(iku)
14,𒑘,1(EŠE₃),21600.0,sq_meter,area,1(EŠE₃),1(EŠE3),1(eše₃),1(eše3)


In [ ]:
copy_new_df = new_df.loc[:, :]

In [ ]:
# Function to extract the number or fraction
def extract_number(value):
    pattern = r'(\d+(?:/\d+)?)(?=\s|\(|$)'
    match = re.search(pattern, value)
    return match.group(1) if match else None

In [ ]:
def process_unit_tr_column(df, column, number):

    df_copy = df.loc[:, :]

    lista = list(df_copy[column].str.split(' '))
    first_value_lista = []
    second_value_lista = []
    third_value_lista = []
    fourth_value_lista = []

    for value in lista:
        if len(value) == 2 or len(value) == 3:
            try:
                number_value = float(value[0])
                first_value_lista.append(number_value)
                second_value_lista.append(value[1])
                third_value_lista.append("NaN")
                fourth_value_lista.append("NaN")
            except ValueError:
                first_value_lista.append("NaN")
                second_value_lista.append(value[0])
                third_value_lista.append(value[1])
                fourth_value_lista.append("NaN")
        elif len(value) == 1:
            first_value_lista.append("NaN")
            second_value_lista.append(value[0])
            third_value_lista.append("NaN")
            fourth_value_lista.append("NaN")
        elif len(value) == 4:
            first_value_lista.append(value[0])
            second_value_lista.append(value[1])
            third_value_lista.append("NaN")
            fourth_value_lista.append(value[3])

    df_copy['first_value'] = df_copy[column].apply(extract_number) # the quantity number
    df_copy['second_value'] = second_value_lista # the unit is used to describe the amount
    df_copy['third_value'] = third_value_lista # an alternative option for the unit
    df_copy['fourth_value'] = fourth_value_lista

    return df_copy

In [ ]:
unit_tr_df = process_unit_tr_column(copy_new_df, 'unit_tr', 5)
unit_tr_unicode_df = process_unit_tr_column(copy_new_df, 'unit_tr_unicode', 5)
unit_tr_digit_df= process_unit_tr_column(copy_new_df, 'unit_tr_digit', 6)
unit_tr_unicode_lower_df = process_unit_tr_column(copy_new_df, 'unit_tr_unicode_lower', 7)
unit_tr_digit_lower_df = process_unit_tr_column(copy_new_df, 'unit_tr_digit_lower', 8)

In [ ]:
def number_unit_dict(df, number_column, unit_column):
  number_dict = {}
  df = df.copy()
  # creates a new unit dictionary for each unique number
  # the NaN key in the units dictionary refers to the count and thus no specificunit
  for index, row in df.iterrows():

    number_dict_key = row[number_column]

    #unit dictionary for a given number
    unit_dict_key = row[unit_column]

    unit_dict_value = row['nr']

    try:
      unit_dict = number_dict[number_dict_key]

    except KeyError:
      number_dict[number_dict_key] = {}
      unit_dict = number_dict[number_dict_key]

    unit_dict[unit_dict_key] = unit_dict_value

  return number_dict

In [ ]:
unit_dict1 = number_unit_dict(unit_tr_df, 'second_value', 'third_value')
unit_dict2 = number_unit_dict(unit_tr_unicode_df, 'second_value', 'third_value')
unit_dict3 = number_unit_dict(unit_tr_digit_df, 'second_value', 'third_value')
unit_dict4 = number_unit_dict(unit_tr_unicode_lower_df , 'second_value', 'third_value')
unit_dict5 = number_unit_dict(unit_tr_digit_lower_df, 'second_value', 'third_value')
combined_dict = {**unit_dict1, **unit_dict2, **unit_dict3, **unit_dict4, **unit_dict5}

In [ ]:
combined_dict

{'GANA₂': {'NaN': 3600.0},
 'GAN₂': {'NaN': 3600.0},
 'ŠE': {'NaN': 0.05},
 'GIN₂': {'NaN': 8.3},
 'SAR': {'NaN': 36.0},
 '1/8(IKU)': {'NaN': 450.0},
 '1/4(IKU)': {'NaN': 900.0},
 '1/2(IKU)': {'NaN': 1800.0},
 '1(IKU)': {'NaN': 3600.0},
 '2(IKU)': {'NaN': 7200.0},
 '3(IKU)': {'NaN': 10800.0},
 '3(IKU)@v': {'NaN': 10800.0},
 '4(IKU)': {'NaN': 14400.0},
 '5(IKU)': {'NaN': 18000.0},
 '1(EŠE₃)': {'NaN': 21600.0},
 '2(EŠE₃)': {'NaN': 43200.0},
 '1(BUR₃)': {'NaN': 64800.0},
 '2(BUR₃)': {'NaN': 129600.0},
 '3(BUR₃)': {'NaN': 194400.0},
 '3(BUR₂)@v': {'NaN': 194400.0},
 '4(BUR₃)': {'NaN': 259200.0},
 '5(BUR₃)': {'NaN': 324000.0},
 '6(BUR₃)': {'NaN': 388800.0},
 '7(BUR₃)': {'NaN': 452600.0},
 '8(BUR₃)': {'NaN': 518400.0},
 '9(BUR₃)': {'NaN': 583200.0},
 '1(BURʾU)': {'NaN': 648000.0},
 '2(BURʾU)': {'NaN': 1296000.0},
 '3(BURʾU)': {'NaN': 1944000.0},
 '3(BURʾU)@v': {'NaN': 1944000.0},
 '4(BURʾU)': {'NaN': 2592000.0},
 '5(BURʾU)': {'NaN': 3240000.0},
 '1(ŠAR₂)': {'NaN': '3600', 'GUN': 108000000.0,

In [ ]:
unique_units_from_dict = pd.concat([
    unit_tr_unicode_df['third_value'],
    unit_tr_digit_df['third_value'],
    unit_tr_unicode_lower_df['third_value'],
    unit_tr_digit_lower_df['third_value']
]).unique().tolist()
unique_units_from_dict

['NaN',
 'MA.NA',
 'GUN',
 'GUR',
 'GUR₇',
 'ŠU.SI',
 'ŠE',
 'ŠĒ',
 'GIN₂',
 'GUR7',
 'GIN2',
 'ma.na',
 'gun',
 'gur',
 'gur₇',
 'šu.si',
 'še',
 'šē',
 'gin₂',
 'gur7',
 'gin2']

Save Dictionary and Unique_units_from_dict

In [ ]:
import pickle

# Saved to cuneiform_dict.pkl in Cuneiform Dictionary file

path = "/content/drive/MyDrive/FactGrid Cuneiform (AWCA)/Econ/Data/weights_measures.pkl"
# Save the list and dictionary to a file
with open(path, 'wb') as f:
    pickle.dump((unique_units_from_dict, combined_dict), f)